# Deploying RF-DETR Model to SageMaker Using PyTorch

This notebook demonstrates end-to-end deployment of the RF-DETR object detection model to Amazon SageMaker using PyTorch. The workflow includes using the pre-trained rf-detr-large checkpoint, creating custom inference code, packaging the model artifact, and deploying to a real-time SageMaker endpoint with GPU acceleration.

This Notebook has been successfully tested on a Mac in VSCode.


## Load Required Packages


In [ ]:
%%sh

brew install python@3.12

python3.12 -m pip install virtualenv --break-system-packages -Uq
python3.12 -m venv .venv
source .venv/bin/activate

In [ ]:
%%sh

# Use this version of pip to avoid compatibility issues
python3.12 -m pip install pip==24.0 -q

In [ ]:
%%sh

# Uninstall any 3.x versions of SageMaker
python3.12 -m pip uninstall sagemaker sagemaker-core sagemaker-mlflow sagemaker-mlops sagemaker-schema-inference-artifacts sagemaker-serve sagemaker-train -q -y

python3.12 -m pip install -r requirements-deploy.txt -q
python3.12 -m pip list | grep sagemaker

## Initialize SageMaker Configuration


In [ ]:
import os

# CHANGE ME: Set to your existing Sagemaker Execution Role ARN
os.environ["SAGEMAKER_ROLE_ARN"] = (
    "arn:aws:iam::<your_account_id>:role/service-role/AmazonSageMaker-ExecutionRole-<your_role_arn>"
)

In [ ]:
import os
import boto3

# Create low-level clients
session = boto3.Session()
region = session.region_name or "us-east-1"
sts = session.client("sts", region_name=region)
iam = session.client("iam", region_name=region)
sagemaker_client = session.client("sagemaker", region_name=region)

# Emulate SageMaker SDK's default bucket convention if you like
role_arn = os.environ.get("SAGEMAKER_ROLE_ARN")
account_id = sts.get_caller_identity()["Account"]
default_bucket = f"sagemaker-{region}-{account_id}"

print(f"Account ID: {account_id}")
print(f"RoleArn: {role_arn}")
print(f"Region: {region}")
print(f"Default S3 Bucket: {default_bucket}")

## Download Model and Configure

In [ ]:
VARIANTS = {
    "nano": {"file": "rf-detr-nano.pth", "type": "rfdetr-nano", "resolution": "384"},
    "small": {"file": "rf-detr-small.pth", "type": "rfdetr-small", "resolution": "512"},
    "medium": {
        "file": "rf-detr-medium.pth",
        "type": "rfdetr-medium",
        "resolution": "576",
    },
    "base": {"file": "rf-detr-base.pth", "type": "rfdetr-base", "resolution": "560"},
    "large": {"file": "rf-detr-large.pth", "type": "rfdetr-large", "resolution": "560"},
}

In [ ]:
%%time

from rfdetr import RFDETRLarge

# Download the model 1x
model = RFDETRLarge()
print(model.model_config)

## Compress and Copy Model Model Artifacts to Amazon S3


In [ ]:
# various settings
artifact_path = "model.tar.gz"
key_prefix = "pytorch-rf-detr"

In [ ]:
%%time

import tarfile
import os

# Choose the weight file you want to package (one of the downloaded files)
model_name = VARIANTS["large"]["file"]
weights_path = os.path.join(os.getcwd(), model_name)
if not os.path.exists(weights_path):
    raise FileNotFoundError(f"Weights not found: {weights_path}")

with tarfile.open(artifact_path, "w:gz") as tar:
    tar.add(weights_path, arcname=model_name)

In [ ]:
%%time

import os
import boto3

s3 = boto3.client("s3")

file_name = os.path.basename(artifact_path)
s3_key = f"{key_prefix}/{file_name}"

# Takes 25-30 seconds on average
s3.upload_file(artifact_path, default_bucket, s3_key)

model_s3_path = f"s3://{default_bucket}/{s3_key}"
print(f"Uploaded model artifact to: {model_s3_path}")

## Create the SageMaker Real-Time Endpoint


In [ ]:
%%time

from sagemaker.pytorch import PyTorchModel
from sagemaker.deserializers import JSONDeserializer
from datetime import datetime, timezone
import sagemaker


# Reuse session from upload cell or create new one
if "session" not in globals():
    session = sagemaker.Session()

print(f"Using model_data: {model_data}")

image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-inference:2.6.0-gpu-py312-cu124-ubuntu22.04-sagemaker-v1.56"

pytorch_model = PyTorchModel(
    model_data=model_s3_path,
    role=role_arn,
    image_uri=image_uri,
    entry_point="inference.py",
    source_dir="code",
    env={
        "RFDETR_VARIANT": "large",
        "RFDETR_CONF": "0.25",
    },
)

instance_type = "ml.g4dn.xlarge"

endpoint_name = (
    VARIANTS["large"]["type"]
    + "-pytorch-"
    + str(datetime.now(timezone.utc).strftime("%Y-%m-%d-%H-%M-%S-%f"))
)
os.environ["SAGEMAKER_ENDPOINT_NAME"] = endpoint_name
print(f"Deploying to endpoint: {endpoint_name}")

# Should take about 7-8 minutes to deploy
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    deserializer=JSONDeserializer(),
)

## Real-time Inference

Perform real-time inference on the sample image directory and display the results with bounding box visualization. We are not passing any additional optional parameters.

In [ ]:
endpoint_name = os.environ.get("SAGEMAKER_ENDPOINT_NAME", "<your_endpoint_name>")
rt = boto3.client("sagemaker-runtime")

In [ ]:
%%time

from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import json
import random
import glob
import os


base_dir = "sample_images"
out_dir = "sample_images_output"
os.makedirs(out_dir, exist_ok=True)

image_paths = sorted(
    glob.glob(os.path.join(base_dir, "*.jpg"))
    + glob.glob(os.path.join(base_dir, "*.jpeg"))
    + glob.glob(os.path.join(base_dir, "*.png"))
)

if not image_paths:
    raise FileNotFoundError(f"No images found in {base_dir}")

print(f"Found {len(image_paths)} images in {base_dir}")

font = ImageFont.load_default(size=24)
IMG_SIZE = int(VARIANTS["large"]["resolution"])


def resize_long_side(image: Image.Image, max_size: int = 640) -> Image.Image:
    w, h = image.size
    long_side = max(w, h)
    if long_side <= max_size:
        return image  # no upscaling
    scale = max_size / float(long_side)
    new_w, new_h = int(w * scale), int(h * scale)
    return image.resize((new_w, new_h), Image.Resampling.LANCZOS)


for image_path in image_paths:
    try:
        orig_image = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Skipping unreadable image: {image_path} - {e}")
        continue

    # Downscale client-side: long side = 560, keep aspect ratio
    send_image = resize_long_side(orig_image, IMG_SIZE)

    buffer = BytesIO()
    send_image.save(buffer, format="JPEG", quality=90)
    payload = buffer.getvalue()

    # Send to endpoint
    response = rt.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="image/jpeg",
        Body=payload,
    )
    result = json.loads(response["Body"].read().decode())
    # print(json.dumps(result, indent=2))

    # ASSUMPTION: boxes are in the coordinates of send_image
    draw = ImageDraw.Draw(orig_image)

    send_w, send_h = send_image.size
    orig_w, orig_h = orig_image.size
    x_ratio = orig_w / send_w
    y_ratio = orig_h / send_h

    for det in result.get("detections", []):
        x1, y1, x2, y2 = det["box"]
        conf = det["confidence"]
        label = det["label"]

        # Scale from send_image coords back to original
        x1, x2 = int(x_ratio * x1), int(x_ratio * x2)
        y1, y2 = int(y_ratio * y1), int(y_ratio * y2)

        # color ranges: 10, 255 full, 10-100 dark, 150-225 light
        color = (
            random.randint(10, 225),
            random.randint(10, 225),
            random.randint(10, 225),
        )

        draw.rectangle([(x1, y1), (x2, y2)], outline=color, width=3)

        text = f"{label} ({int(conf * 100)}%)"
        draw.text((x1, max(0, y1 - 30)), text, fill=color, font=font)

    base_name = os.path.basename(image_path)
    name, ext = os.path.splitext(base_name)
    out_path = os.path.join(out_dir, f"{name}_detected{ext}")
    orig_image.save(out_path, quality=95)
    print(f"Saved: {out_path}")

print(f"Done. {len(image_paths)} images processed; results in: {out_dir}")

### Real-time Inference with JSON Format and Dynamic Confidence Threshold

The inference endpoint supports a JSON request format that allows you to dynamically specify the confidence threshold per request. This is useful for:

- Testing different confidence thresholds without redeploying
- Adjusting sensitivity based on use case
- A/B testing different threshold values

**JSON Request Format:**

```json
{
  "image": "base64_encoded_image_data",
  "confidence": 0.35 // Optional: defaults to 0.25 if not provided
}
```

The example below demonstrates how to use the JSON format with different confidence thresholds.


In [ ]:
%%time

from PIL import Image
from io import BytesIO
import json
import base64
import glob
import os


# Load a sample image
base_dir = "sample_images"
image_paths = glob.glob(os.path.join(base_dir, "*.jpg"))[:1]  # Test with first image
IMG_SIZE = int(VARIANTS["large"]["resolution"])

if not image_paths:
    raise FileNotFoundError(f"No images found in {base_dir}")

image_path = image_paths[0]
print(f"Testing with image: {image_path}")

# Load and prepare image
orig_image = Image.open(image_path).convert("RGB")


# Downscale client-side: long side = 560, keep aspect ratio
def resize_long_side(image: Image.Image, max_size: int = 560) -> Image.Image:
    w, h = image.size
    long_side = max(w, h)
    if long_side <= max_size:
        return image
    scale = max_size / float(long_side)
    new_w, new_h = int(w * scale), int(h * scale)
    return image.resize((new_w, new_h), Image.Resampling.LANCZOS)


send_image = resize_long_side(orig_image, IMG_SIZE)

buffer = BytesIO()
send_image.save(buffer, format="JPEG", quality=90)
image_bytes = buffer.getvalue()

# Test with different confidence thresholds
confidence_thresholds = [0.25, 0.5, 0.90, 0.94]

print(f"\nTesting with different confidence thresholds:\n")

for confidence in confidence_thresholds:
    # Create JSON payload with base64-encoded image and confidence
    payload = {
        "image": base64.b64encode(image_bytes).decode("utf-8"),
        "confidence": confidence,
    }

    # Send to endpoint
    response = rt.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
    )
    result = json.loads(response["Body"].read().decode())

    detection_count = result.get("metadata", {}).get("count", 0)
    inference_time = result.get("metadata", {}).get("inference_time_ms", "N/A")

    print(
        f"Confidence: {confidence:.2f} | Detections: {detection_count:2d} | Inference time: {inference_time} ms"
    )

    # Print first detection for reference
    if result.get("detections"):
        first_det = result["detections"][0]
        print(
            f"  → First detection: {first_det['label']} ({first_det['confidence']:.3f})"
        )

print(
    f"\nNote: Higher confidence thresholds typically result in fewer but more confident detections."
)

### Advanced Filtering: Classes, Max Detections, and Minimum Box Area

In addition to dynamic confidence thresholds, the endpoint supports powerful post-processing filters:

1. **`classes`**: Filter to specific object classes (e.g., only "person", "car"). Set to `null` to return all classes.
2. **`max_detections`**: Limit the number of detections returned (returns top N by confidence)
3. **`min_box_area`**: Filter out small detections below a minimum bounding box area (in pixels²)

These filters are applied after inference, so they don't affect inference speed but can significantly reduce response payload size.

**Example JSON Request:**

```json
{
  "image": "base64_encoded_image",
  "confidence": 0.3,
  "classes": ["person", "car"], // Only return people and cars (or null for all)
  "max_detections": 10, // Return top 10 detections only
  "min_box_area": 1000 // Filter out boxes smaller than 1000 px²
}
```


In [ ]:
%%time

from PIL import Image
from io import BytesIO
import json
import base64
import glob
import os


# Load a sample image
base_dir = "sample_images"
image_paths = glob.glob(os.path.join(base_dir, "*.jpg"))[:1]
IMG_SIZE = int(VARIANTS["large"]["resolution"])

if not image_paths:
    raise FileNotFoundError(f"No images found in {base_dir}")

image_path = image_paths[0]
print(f"Testing with image: {image_path}\n")

# Load and prepare image
orig_image = Image.open(image_path).convert("RGB")


def resize_long_side(image: Image.Image, max_size: int = 560) -> Image.Image:
    w, h = image.size
    long_side = max(w, h)
    if long_side <= max_size:
        return image
    scale = max_size / float(long_side)
    new_w, new_h = int(w * scale), int(h * scale)
    return image.resize((new_w, new_h), Image.Resampling.LANCZOS)


send_image = resize_long_side(orig_image, IMG_SIZE)

buffer = BytesIO()
send_image.save(buffer, format="JPEG", quality=90)
image_bytes = buffer.getvalue()
image_b64 = base64.b64encode(image_bytes).decode("utf-8")

# Example 1: Return all classes (no class filtering)
print("=" * 80)
print("Example 1: Return all classes (classes=null)")
print("=" * 80)
payload = {
    "image": image_b64,
    "confidence": 0.25,
    "classes": None,  # Explicitly request all classes
}
response = rt.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload),
)
result1 = json.loads(response["Body"].read().decode())
print(f"Detections: {result1['metadata']['count']}")
print(f"Classes found: {list(set([d['label'] for d in result1['detections']]))}")
print()

# Example 2: Filter to specific classes only
print("=" * 80)
print("Example 2: Filter to specific classes (scissors, bottle, knife)")
print("=" * 80)
payload = {
    "image": image_b64,
    "confidence": 0.25,
    "classes": ["scissors", "bottle", "knife"],
}
response = rt.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload),
)
result2 = json.loads(response["Body"].read().decode())
print(f"Detections: {result2['metadata']['count']}")
print(f"Classes found: {list(set([d['label'] for d in result1['detections']]))}")
if result2["metadata"].get("applied_filters"):
    print(f"Applied filters: {result2['metadata']['applied_filters']}")
print()

# Example 3: Limit to top 5 detections
print("=" * 80)
print("Example 3: Limit to top 5 detections by confidence")
print("=" * 80)
payload = {
    "image": image_b64,
    "confidence": 0.2,  # Lower confidence to get more candidates
    "max_detections": 5,
}
response = rt.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload),
)
result3 = json.loads(response["Body"].read().decode())
print(f"Detections returned: {result3['metadata']['count']}")
if result3["metadata"].get("applied_filters"):
    print(f"Applied filters: {result3['metadata']['applied_filters']}")
if result3["detections"]:
    print(
        f"Top detection: {result3['detections'][0]['label']} ({result3['detections'][0]['confidence']:.3f})"
    )
print()

# Example 4: Filter by minimum box area
print("=" * 80)
print("Example 4: Filter out small detections (min_box_area=8000 px²)")
print("=" * 80)
payload = {"image": image_b64, "confidence": 0.25, "min_box_area": 8000}
response = rt.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload),
)
result4 = json.loads(response["Body"].read().decode())
print(f"Detections (large objects only): {result4['metadata']['count']}")
if result4["detections"]:
    for det in result4["detections"]:
        print(
            f"  - {det['label']}: {det['area']:.1f} px² (conf: {det['confidence']:.3f})"
        )
if result4["metadata"].get("applied_filters"):
    print(f"Applied filters: {result4['metadata']['applied_filters']}")
print()

# Example 5: Combine all filters
print("=" * 80)
print("Example 5: Combine all filters")
print("=" * 80)
payload = {
    "image": image_b64,
    "confidence": 0.2,
    "classes": ["scissors", "bottle", "knife"],
    "max_detections": 3,
    "min_box_area": 1000,
}
response = rt.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload),
)
result5 = json.loads(response["Body"].read().decode())
print(f"Final detections: {result5['metadata']['count']}")
print(f"Applied filters: {result5['metadata'].get('applied_filters', {})}")
for det in result5["detections"]:
    print(
        f"  - {det['label']}: area={det['area']:.1f}px², conf={det['confidence']:.3f}"
    )

print("\n" + "=" * 80)
print("Summary: Filters allow you to customize responses without redeploying!")
print("=" * 80)

## Video Object Detection with Amazon SageMaker


### Simple Video Object Detections

In [ ]:
import os
import base64
from io import BytesIO
import numpy as np
from PIL import Image
import supervision as sv

SOURCE_VIDEO_PATH = "sample_images/sample_video.mp4"
TARGET_VIDEO_PATH = "sample_images_output/sample_video_annotated.mp4"
IMG_SIZE = int(VARIANTS["large"]["resolution"])


mask_annotator = sv.MaskAnnotator(
    opacity=0.4,
)
box_annotator = sv.BoxAnnotator(
    thickness=2,
)
label_annotator = sv.LabelAnnotator(
    # text_scale=0.4,
    # text_padding=8,
    # text_thickness=0,
    smart_position=True,  # <– try to avoid overlapping labels
)


def resize_long_side(image: Image.Image, max_size: int = 560) -> Image.Image:
    w, h = image.size
    if max(w, h) <= max_size:
        return image
    scale = max_size / float(max(w, h))
    new_w, new_h = int(w * scale), int(h * scale)
    return image.resize((new_w, new_h), Image.Resampling.LANCZOS)


def callback(frame: np.ndarray, index: int) -> np.ndarray:
    if index % 50 == 0:
        print(f"[cb] frame {index}, shape={frame.shape}")

    h_orig, w_orig = frame.shape[:2]

    # BGR -> RGB PIL
    rgb = frame[:, :, ::-1]
    pil_frame = Image.fromarray(rgb)

    # Resize for inference
    send_frame = resize_long_side(pil_frame, IMG_SIZE)
    w_inf, h_inf = send_frame.size

    # Scale from inference coords -> original coords
    scale_x = w_orig / w_inf
    scale_y = h_orig / h_inf

    # JPEG buffer
    buf = BytesIO()
    send_frame.save(buf, format="JPEG", quality=90)

    payload = {
        "image": base64.b64encode(buf.getvalue()).decode("utf-8"),
        "confidence": 0.25,
    }

    response = rt.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
    )
    result = json.loads(response["Body"].read().decode())

    detections_raw = result["detections"]

    if not detections_raw:
        return frame

    # RF‑DETR: box = [x1, y1, x2, y2] in resized space
    boxes_orig = []
    for d in detections_raw:
        x1, y1, x2, y2 = d["box"]
        boxes_orig.append(
            [
                x1 * scale_x,
                y1 * scale_y,
                x2 * scale_x,
                y2 * scale_y,
            ]
        )

    detections = sv.Detections(
        xyxy=np.array(boxes_orig),
        class_id=np.array([d["class_id"] for d in detections_raw]),
        confidence=np.array([d["confidence"] for d in detections_raw]),
    )

    labels = [f"{d['label']} {d['confidence'] * 100:.1f}%" for d in detections_raw]

    annotated = box_annotator.annotate(
        scene=frame.copy(),
        detections=detections,
    )
    annotated = label_annotator.annotate(
        scene=annotated,
        detections=detections,
        labels=labels,
    )

    return annotated


print("Starting video processing...")
sv.process_video(
    source_path=SOURCE_VIDEO_PATH,
    target_path=TARGET_VIDEO_PATH,
    callback=callback,
    show_progress=True,  # supported in recent versions [web:22][web:26]
)
print("Processing complete.")
print("Final file size (bytes):", os.path.getsize(TARGET_VIDEO_PATH))